# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [10]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=College;Trusted_Connection=yes;')


In [11]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students",conn) 
print(students)

     StudentId FirstName   LastName Gender
0        162.0    Elmira    Holston      F
1        154.0    Sierra  Van Vorst      F
2         28.0     Mycah    Bingham      F
3         54.0      Alia      Jones      F
4         31.0    Chanda   Fillmore      F
..         ...       ...        ...    ...
275      246.0   Preston    Sprouse      M
276       55.0   Dmitriy       Gage      M
277      218.0    Skyler     Norris      M
278      269.0     Tyler    Raymond      M
279      228.0     Dylan      Davis      M

[280 rows x 4 columns]


# Questions
 
 
## Q1. Count the number of students on each department

In [13]:
students.groupby('Department').agg(['count'])

KeyError: 'Department'

## Q2. How many students have each course of the English department and the total number of students in the department?

## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

## Q5. For which courses the percentage of male/female students is over 70%?

## Q6. For each department, how many students passed with a grades over 80?

## Q7. For each department, how many students passed with a grades under 60?

## Q8. Rate the teachers by their average student's grades (in descending order).

## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).